In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from helper import to_df

In [2]:
def uniform_scheme(node_id: int, nodes_df: pd.DataFrame) -> float:
    """ Assign uniform weights """
    return [1 / nodes_df.loc[node_id, 'NKids'] for i in range(nodes_df.loc[node_id, 'NKids'])]

In [3]:
%%time
tree = './trees/golomb_6.sqlite'
nodes_df = to_df(tree, 'nodes').set_index('NodeID')
print(nodes_df.shape)

nodes_df['NodeWeight'] = 0
nodes_df.loc[0, 'NodeWeight'] = 1 # root node has weight 1
i = 0

# propogate weights down
while i < nodes_df.shape[0] - 1:

    par_weight = nodes_df.loc[i, 'NodeWeight']
    num_kids = nodes_df.loc[i, 'NKids']
    if num_kids > 0:
        weights = uniform_scheme(i, nodes_df)
        assert len(weights) == num_kids
        nodes_df.loc[nodes_df['ParentID'] == i, 'NodeWeight'] = par_weight * \
                                nodes_df[nodes_df['ParentID'] == i]['Alternative'].apply(lambda x: weights[x]) 
    i += 1

(190, 5)
CPU times: user 113 ms, sys: 2.11 ms, total: 115 ms
Wall time: 135 ms


In [ ]:
dfs_ordering = [0]
visited = set([0])
boundary = nodes_df[nodes_df['ParentID'] == 0].sort_values('Alternative', ascending=False).index.to_list()

# run simulated dfs on tree
while len(boundary) > 0:
    nxt = boundary.pop()
    # tree, so nxt will not be in either visited or boundary
    dfs_ordering.append(nxt)
    boundary.extend(nodes_df[nodes_df['ParentID'] == nxt].sort_values('Alternative', ascending=False).index.to_list())





In [ ]:
cumulative = nodes_df.set_index(dfs_ordering)\
                    [nodes_df['Status'].isin({1, 0, 3})]['NodeWeight']\
                    .cumsum()\
                    .reindex(list(range(1, nodes_df.shape[0]+1)), method='ffill')\
                    .fillna(0)

x = range(1, nodes_df.shape[0] + 1)
plt.plot(pd.Series(x) / nodes_df.shape[0], label='Ground Truth')
plt.plot(cumulative, label='Uniform weight')
plt.legend()
plt.show()